In [15]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder

class HAM10000Dataset(Dataset):
    def __init__(self, csv_file, img_dirs, transform=None, device='cuda'):
        self.data = pd.read_csv(csv_file)
        self.img_dirs = img_dirs
        self.transform = transform
        self.device = device
        
        # Encode labels
        self.label_encoder = LabelEncoder()
        self.data['encoded_label'] = self.label_encoder.fit_transform(self.data['dx'])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]['image_id'] + '.jpg'
        for img_dir in self.img_dirs:
            img_path = os.path.join(img_dir, img_name)
            if os.path.exists(img_path):
                image = Image.open(img_path).convert('RGB')
                if self.transform:
                    image = self.transform(image)
                label = self.data.iloc[idx]['encoded_label']
                return image, label
        raise FileNotFoundError(f"Image {img_name} not found in directories {self.img_dirs}")

class FastGANGenerator(nn.Module):
    def __init__(self, latent_dim=100, img_channels=3, num_classes=7, device='cuda'):
        super(FastGANGenerator, self).__init__()
        self.device = device
        
        # Condition embedding
        self.label_embedding = nn.Embedding(num_classes, num_classes).to(device)
        
        # Generator architecture with conditional input
        self.model = nn.Sequential(
            nn.Linear(latent_dim + num_classes, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            
            nn.Linear(1024, img_channels * 64 * 64),
            nn.Tanh()
        ).to(device)
    
    def forward(self, z, labels):
        # Ensure z and labels are on the correct device
        z = z.to(self.device)
        labels = labels.to(self.device)
        
        # Embed labels
        label_embed = self.label_embedding(labels)
        
        # Concatenate noise and label embedding
        conditional_input = torch.cat([z, label_embed], dim=1)
        
        # Generate images
        img = self.model(conditional_input)
        img = img.view(img.size(0), 3, 64, 64)
        return img

class FastGANDiscriminator(nn.Module):
    def __init__(self, img_channels=3, num_classes=7, device='cuda'):
        super(FastGANDiscriminator, self).__init__()
        self.device = device
        
        # Label embedding
        self.label_embedding = nn.Embedding(num_classes, num_classes).to(device)
        
        self.model = nn.Sequential(
            nn.Linear(img_channels * 64 * 64 + num_classes, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(512, 1),
            nn.Sigmoid()
        ).to(device)
    
    def forward(self, img, labels):
        # Ensure img and labels are on the correct device
        img = img.to(self.device)
        labels = labels.to(self.device)
        
        # Flatten image
        img_flat = img.view(img.size(0), -1)
        
        # Embed labels
        label_embed = self.label_embedding(labels)
        
        # Concatenate image and label embedding
        conditional_input = torch.cat([img_flat, label_embed], dim=1)
        
        # Compute validity
        validity = self.model(conditional_input)
        return validity

def train_fastgan(generator, discriminator, dataloader, num_epochs=1):
    """
    Train Conditional FASTGAN
    """
    device = generator.device
    
    # Optimizers
    g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    # Loss functions
    adversarial_loss = nn.BCELoss()
    
    for epoch in range(num_epochs):
        for real_images, labels in dataloader:
            real_images = real_images.to(device)
            labels = labels.to(device)
            batch_size = real_images.size(0)
            
            # Adversarial ground truths
            valid = torch.ones(batch_size, 1).to(device)
            fake = torch.zeros(batch_size, 1).to(device)
            
            # Train Generator
            g_optimizer.zero_grad()
            
            # Sample noise and labels
            z = torch.randn(batch_size, 100).to(device)
            gen_labels = torch.randint(0, len(generator.label_embedding.weight), (batch_size,)).to(device)
            
            # Generate images
            generated_images = generator(z, gen_labels)
            
            # Generator loss
            g_loss = adversarial_loss(discriminator(generated_images, gen_labels), valid)
            
            g_loss.backward()
            g_optimizer.step()
            
            # Train Discriminator
            d_optimizer.zero_grad()
            
            # Real images loss
            real_loss = adversarial_loss(discriminator(real_images, labels), valid)
            
            # Fake images loss
            fake_loss = adversarial_loss(discriminator(generated_images.detach(), gen_labels), fake)
            
            # Total discriminator loss
            d_loss = (real_loss + fake_loss) / 2
            
            d_loss.backward()
            d_optimizer.step()
        
        # Print epoch statistics
        print(f"Epoch [{epoch+1}/{num_epochs}], G Loss: {g_loss.item():.4f}, D Loss: {d_loss.item():.4f}")
    
    return generator

def main():
    # Determine device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Set random seed for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    
    # Transforms
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    # 1. Prepare HAM10000 Dataset
    csv_file = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv'
    img_dirs = ['/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1', '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2']
    
    # Create dataset and data loader
    dataset = HAM10000Dataset(csv_file, img_dirs, transform=transform, device=device)
    
    # Get number of classes
    num_classes = len(dataset.label_encoder.classes_)
    print("Unique Classes:", dataset.label_encoder.classes_)
    print("Number of Classes:", num_classes)
    
    # Create data loader
    batch_size = 64
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    
    # 2. Setup FASTGAN Configuration
    generator = FastGANGenerator(num_classes=num_classes, device=device)
    discriminator = FastGANDiscriminator(num_classes=num_classes, device=device)
    
    # 3. Train FASTGAN and Generate 1000 Synthetic Images
    trained_generator = train_fastgan(generator, discriminator, data_loader)
    
    # Generate synthetic images
    os.makedirs('synthetic_images', exist_ok=True)
    
    # Generate images for each class
    synthetic_images_by_class = {}
    
    with torch.no_grad():
        for class_idx in range(num_classes):
            # Generate 100 images for each class
            z = torch.randn(100, 100).to(device)
            class_labels = torch.full((100,), class_idx, dtype=torch.long).to(device)
            
            synthetic_images = trained_generator(z, class_labels)
            
            # Save images
            synthetic_images_by_class[class_idx] = synthetic_images.cpu()
            
            # Save images to disk
            class_name = dataset.label_encoder.inverse_transform([class_idx])[0]
            class_dir = os.path.join('synthetic_images', class_name)
            os.makedirs(class_dir, exist_ok=True)
            
            for i, img in enumerate(synthetic_images):
                save_path = os.path.join(class_dir, f'synthetic_image_{i}.png')
                save_image((img * 0.5 + 0.5), save_path)
    
    print("Synthetic image generation and filtering complete!")

if __name__ == "__main__":
    main()

Using device: cuda
Unique Classes: ['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']
Number of Classes: 7
Epoch [1/1], G Loss: 3.6399, D Loss: 0.2509
Synthetic image generation and filtering complete!
